In [ ]:
import cv2
import mediapipe as mp
import time
import pandas as pd
import numpy as np 
import os
import pickle
from datetime import datetime, time as dt_time
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import optuna
from optuna.samplers import GridSampler

In [1]:
fm = mp.solutions.face_mesh
model = fm.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8,
)


vid = cv2.VideoCapture(1) 


regist = int(input("How many faces to register? "))
final_data = []

for y in range(regist):
    name = input(f"\nEnter name for person {y + 1}: ")
    print(f"\nCapturing 300 samples for '{name}' in 3 seconds...")
    time.sleep(3)

    sample_count = 0
    required_samples = 400

    while sample_count < required_samples:
        success, frame = vid.read()
        if not success:
            print("Camera error.")
            break

        
        frgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        output = model.process(frgb)

        
        if output.multi_face_landmarks:
            
            mp.solutions.drawing_utils.draw_landmarks(
                image=frame,
                landmark_list=output.multi_face_landmarks[0],
                connections=fm.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style()
            )

            
            face = []
            for lm in output.multi_face_landmarks[0].landmark:
                face.extend([lm.x, lm.y, lm.z])
            face.append(name)
            final_data.append(face)
            sample_count += 1
            #print(f"Captured sample {sample_count}/{required_samples} for {name}")
            time.sleep(0.09)

        
        cv2.imshow("Face Registration", frame)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Early quit.")
            vid.release()
            cv2.destroyAllWindows()
            exit()

    print(f"Finished capturing for '{name}'")


vid.release()
cv2.destroyAllWindows()

num_features = 478 * 3
columns = list(range(num_features)) + ['label']
df = pd.DataFrame(final_data, columns=columns)


How many faces to register?  8

Enter name for person 1:  bhagyasri



Capturing 300 samples for 'bhagyasri' in 3 seconds...
Finished capturing for 'bhagyasri'



Enter name for person 2:  varshitha



Capturing 300 samples for 'varshitha' in 3 seconds...
Finished capturing for 'varshitha'



Enter name for person 3:  archana


...


Capturing 300 samples for 'nikitha' in 3 seconds...
Finished capturing for 'nikitha'



Enter name for person 7:  kalyani



Capturing 300 samples for 'kalyani' in 3 seconds...
Finished capturing for 'kalyani'



Enter name for person 8:  snehit



Capturing 300 samples for 'snehit' in 3 seconds...
Finished capturing for 'snehit'


In [2]:
df

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,label
0,0.502483,0.566754,-0.042972,0.506401,0.506829,-0.064901,0.501486,0.526603,-0.037018,0.493963,...,0.557310,0.398496,0.030720,0.543912,0.412093,0.030720,0.555628,0.428331,0.030720,bhagyasri
1,0.537355,0.558633,-0.042912,0.536422,0.495425,-0.059814,0.530764,0.519298,-0.035890,0.519930,...,0.572460,0.406428,0.037972,0.562239,0.419280,0.037972,0.572911,0.433037,0.037972,bhagyasri
2,0.534944,0.561215,-0.044920,0.540370,0.496933,-0.058600,0.533350,0.522895,-0.036508,0.525378,...,0.573389,0.419539,0.043543,0.561797,0.431957,0.043543,0.571900,0.447968,0.043543,bhagyasri
3,0.546626,0.575104,-0.047647,0.553915,0.511821,-0.055398,0.546419,0.539787,-0.036732,0.537591,...,0.584105,0.446816,0.050631,0.573186,0.457370,0.050631,0.582549,0.471812,0.050631,bhagyasri
4,0.538473,0.603222,-0.045840,0.542914,0.545084,-0.054496,0.535162,0.571046,-0.035680,0.527939,...,0.576081,0.478401,0.048879,0.565271,0.487726,0.048879,0.573798,0.502639,0.048879,bhagyasri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0.690479,0.566989,-0.038011,0.703302,0.493252,-0.078247,0.697181,0.512757,-0.040838,0.688043,...,0.759246,0.348679,0.016497,0.746824,0.363432,0.016497,0.759158,0.378890,0.016497,snehit
3196,0.704261,0.572899,-0.036287,0.721451,0.491907,-0.075429,0.713896,0.512093,-0.038392,0.705478,...,0.770128,0.346234,0.021364,0.756531,0.361162,0.021364,0.769503,0.378908,0.021364,snehit
3197,0.724405,0.579229,-0.035315,0.737167,0.493981,-0.073956,0.727645,0.513831,-0.037221,0.721379,...,0.780090,0.348493,0.025672,0.765458,0.363901,0.025672,0.779070,0.381395,0.025672,snehit
3198,0.733659,0.568495,-0.036775,0.753595,0.498424,-0.073210,0.740829,0.518139,-0.037917,0.737382,...,0.787057,0.359677,0.033120,0.772920,0.375750,0.033120,0.786407,0.394214,0.033120,snehit


In [3]:
df["label"].value_counts()

label
bhagyasri    400
varshitha    400
archana      400
madhavi      400
sravani      400
nikitha      400
kalyani      400
snehit       400
Name: count, dtype: int64

In [4]:
df.to_csv(r"D:\BHAGYASRI\DATASCIENCE\ml\ats_data.csv", index=False)
print("\nData saved in numbered format to 'ats_data.csv'")


Data saved in numbered format to 'ats_data.csv'


In [12]:
df=pd.read_csv(r"D:\BHAGYASRI\DATASCIENCE\ml\ats_data.csv")

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,label
0,0.502483,0.566754,-0.042972,0.506401,0.506829,-0.064901,0.501486,0.526603,-0.037018,0.493963,...,0.557310,0.398496,0.030720,0.543912,0.412093,0.030720,0.555628,0.428331,0.030720,bhagyasri
1,0.537355,0.558633,-0.042912,0.536422,0.495425,-0.059814,0.530764,0.519298,-0.035890,0.519930,...,0.572460,0.406428,0.037972,0.562239,0.419280,0.037972,0.572911,0.433037,0.037972,bhagyasri
2,0.534944,0.561215,-0.044920,0.540370,0.496933,-0.058600,0.533350,0.522895,-0.036508,0.525378,...,0.573389,0.419539,0.043543,0.561797,0.431957,0.043543,0.571900,0.447968,0.043543,bhagyasri
3,0.546626,0.575104,-0.047647,0.553915,0.511821,-0.055398,0.546419,0.539787,-0.036732,0.537591,...,0.584105,0.446816,0.050631,0.573186,0.457370,0.050631,0.582549,0.471812,0.050631,bhagyasri
4,0.538473,0.603222,-0.045840,0.542914,0.545084,-0.054496,0.535162,0.571046,-0.035680,0.527939,...,0.576081,0.478401,0.048879,0.565271,0.487726,0.048879,0.573798,0.502639,0.048879,bhagyasri


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Columns: 1435 entries, 0 to label
dtypes: float64(1434), object(1)
memory usage: 35.0+ MB


In [15]:
df.dropna(inplace=True)

In [16]:
fv=df.iloc[:,:-1]
cv=df.iloc[:,-1]

In [17]:
fv.shape

(3200, 1434)

In [18]:
cv.shape

(3200,)

In [19]:
final_p_data=[]
for d1 in fv.values:
    md=d1.reshape(478,3)
    centre=md-d1.reshape(478,3)[1]
    distance=np.linalg.norm(d1.reshape(478,3)[33]-d1.reshape(478,3)[263])
    fpd=centre/distance
    final_p_data.append(fpd.flatten())

In [20]:
pd.DataFrame(final_p_data)

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433
0,-0.020843,0.318778,0.116650,0.0,0.0,0.0,-0.026149,0.105192,0.148324,-0.066166,...,0.508659,0.270812,-0.576282,0.508659,0.199540,-0.503955,0.508659,0.261866,-0.417576,0.508659
1,0.005404,0.366218,0.097929,0.0,0.0,0.0,-0.032781,0.138318,0.138611,-0.095553,...,0.566556,0.208797,-0.515631,0.566556,0.149574,-0.441167,0.566556,0.211408,-0.361466,0.566556
2,-0.031860,0.377478,0.080329,0.0,0.0,0.0,-0.041222,0.152452,0.129725,-0.088034,...,0.599803,0.193897,-0.454472,0.599803,0.125823,-0.381551,0.599803,0.185150,-0.287533,0.599803
3,-0.043908,0.381235,0.046692,0.0,0.0,0.0,-0.045160,0.168476,0.112448,-0.098341,...,0.638750,0.181876,-0.391611,0.638750,0.116094,-0.328028,0.638750,0.172501,-0.241027,0.638750
4,-0.027062,0.354297,0.052747,0.0,0.0,0.0,-0.047240,0.158209,0.114667,-0.091258,...,0.629974,0.202125,-0.406374,0.629974,0.136251,-0.349549,0.629974,0.188213,-0.258664,0.629974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,-0.064274,0.369581,0.201670,0.0,0.0,0.0,-0.030681,0.097761,0.187501,-0.076480,...,0.474873,0.280402,-0.724623,0.474873,0.218139,-0.650682,0.474873,0.279957,-0.573205,0.474873
3196,-0.085087,0.400899,0.193746,0.0,0.0,0.0,-0.037394,0.099920,0.183328,-0.079063,...,0.479112,0.240948,-0.721062,0.479112,0.173644,-0.647167,0.479112,0.237854,-0.559330,0.479112
3197,-0.062419,0.416967,0.189006,0.0,0.0,0.0,-0.046575,0.097090,0.179683,-0.077225,...,0.487307,0.209950,-0.711620,0.487307,0.138381,-0.636260,0.487307,0.204958,-0.550689,0.487307
3198,-0.095212,0.334641,0.174004,0.0,0.0,0.0,-0.060968,0.094154,0.168550,-0.077433,...,0.507806,0.159806,-0.662624,0.507806,0.092290,-0.585862,0.507806,0.156700,-0.497682,0.507806


In [21]:
pd.DataFrame(final_p_data).shape

(3200, 1434)

In [23]:
rf_model=RandomForestClassifier(n_estimators=10)
rf_model.fit(final_p_data,cv)

RandomForestClassifier(n_estimators=10)

In [1]:


FACE_DATA_FILE = "attendance_users_data.csv"
ATTENDANCE_FILE = "attendance_log_records.csv"
MODEL_FILE = "trained_face_model.pkl"
FEATURES_FILE = "landmark_features.pkl"

class FaceAttendanceSystem:
    def __init__(self):  # Fixed constructor
        self.SAMPLES = 150
        self.TIMEOUT_SECONDS = 10
        self.CAMERA_INDEX = 1
        self.LOGIN_START = dt_time(9, 0, 0)
        self.LOGIN_END = dt_time(9, 30, 0)
        self.LOGOUT_TIME = dt_time(17, 30, 0)
        self.face_data_file = FACE_DATA_FILE
        self.attendance_file = ATTENDANCE_FILE
        self.model_file = MODEL_FILE
        self.features_file = FEATURES_FILE
        self.active_users = {}
        self.login_times = {}
        self.model = None
        self.features = None

        self.drawing_spec = mp.solutions.drawing_utils.DrawingSpec(thickness=1, circle_radius=1)
        self.face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.8,
            min_tracking_confidence=0.8
        )

        if not os.path.exists(self.attendance_file):
            df = pd.DataFrame(columns=['Name', 'Login Time', 'Logout Time', 'Duration', 'Login Status'])
            df = df.astype(str)
            df.to_csv(self.attendance_file, index=False)

        self.load_model()

    def preprocess(self, landmarks):
        face = np.array([[lm.x, lm.y, lm.z] for lm in landmarks])
        centered = face - face[1]
        distance = np.linalg.norm(face[33] - face[263])
        normalized = centered / distance
        return normalized.flatten()

    def register_user(self, name):
        cap = cv2.VideoCapture(self.CAMERA_INDEX)
        collected_data = []
        sample_count = 0
        print(f"[INFO] Registering: {name}")
        time.sleep(2)

        while sample_count < self.SAMPLES:
            ret, frame = cap.read()
            if not ret:
                break

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = self.face_mesh.process(rgb)

            if result.multi_face_landmarks:
                landmarks = result.multi_face_landmarks[0].landmark
                vector = self.preprocess(landmarks).tolist()
                vector.append(name)
                collected_data.append(vector)
                sample_count += 1

                mp.solutions.drawing_utils.draw_landmarks(
                    frame, result.multi_face_landmarks[0],
                    mp.solutions.face_mesh.FACEMESH_TESSELATION,
                    self.drawing_spec, self.drawing_spec
                )
                cv2.putText(frame, f"{name}: {sample_count}/{self.SAMPLES}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            cv2.imshow("Registering New User", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

        if collected_data:
            columns = list(range(len(collected_data[0]) - 1)) + ['label']
            df = pd.DataFrame(collected_data, columns=columns)
            if os.path.exists(self.face_data_file):
                df.to_csv(self.face_data_file, mode='a', header=False, index=False)
            else:
                df.to_csv(self.face_data_file, index=False)
            self.train_model()
            print(f"[SUCCESS] {name} registered successfully.")
        else:
            print("[ERROR] No data collected.")

    def train_model(self):
        df = pd.read_csv(self.face_data_file)
        X = df.drop('label', axis=1)
        y = df['label']
        self.features = X.columns.tolist()

        search_space = {
            'n_estimators': [50, 100, 150],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
        sampler = GridSampler(search_space)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_categorical('n_estimators', search_space['n_estimators']),
                'max_depth': trial.suggest_categorical('max_depth', search_space['max_depth']),
                'min_samples_split': trial.suggest_categorical('min_samples_split', search_space['min_samples_split']),
                'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', search_space['min_samples_leaf'])
            }
            model = RandomForestClassifier(**params, random_state=42)
            scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')
            return scores.mean()

        study = optuna.create_study(direction='maximize', sampler=sampler)
        study.optimize(objective, n_trials=len(sampler._all_grids))

        best_params = study.best_params
        self.model = RandomForestClassifier(**best_params, random_state=42)
        self.model.fit(X, y)

        pickle.dump(self.model, open(self.model_file, 'wb'))
        pickle.dump(self.features, open(self.features_file, 'wb'))

    def load_model(self):
        if os.path.exists(self.model_file) and os.path.exists(self.features_file):
            self.model = pickle.load(open(self.model_file, 'rb'))
            self.features = pickle.load(open(self.features_file, 'rb'))

    def update_attendance_log(self, name, login_time, logout_time):
        duration = logout_time - login_time
        df = pd.read_csv(self.attendance_file)

        df['Logout Time'] = df['Logout Time'].astype(str)
        df['Duration'] = df['Duration'].astype(str)
        df['Login Status'] = df['Login Status'].astype(str)

        login_status = "Late" if login_time.time() > self.LOGIN_END else "On Time"
        mask = (df['Name'] == name) & ((df['Logout Time'].isna()) | (df['Logout Time'] == 'nan') | (df['Logout Time'] == ''))

        if mask.any():
            df.loc[mask, 'Logout Time'] = logout_time.strftime("%Y-%m-%d %H:%M:%S")
            df.loc[mask, 'Duration'] = str(duration)
            df.loc[mask, 'Login Status'] = login_status
            print(f"[LOGOUT] {name} at {logout_time.strftime('%H:%M:%S')} | Duration: {duration} | Status: {login_status}")
        else:
            print(f"[SKIPPED LOGOUT] No login record found for {name}. Ignored.")

        df.to_csv(self.attendance_file, index=False)

    def mark_login(self, name, current_time):
        self.active_users[name] = current_time
        self.login_times[name] = current_time

        df = pd.read_csv(self.attendance_file)
        login_status = "Late" if current_time.time() > self.LOGIN_END else "On Time"
        new_entry = pd.DataFrame([{ 
            'Name': name,
            'Login Time': current_time.strftime("%Y-%m-%d %H:%M:%S"),
            'Logout Time': '',
            'Duration': '',
            'Login Status': login_status
        }])
        df = pd.concat([df, new_entry], ignore_index=True)
        df.to_csv(self.attendance_file, index=False)
        print(f"[LOGIN] {name} at {current_time.strftime('%H:%M:%S')} | Status: {login_status}")

    def start_attendance(self):
        if self.model is None or self.features is None:
            print("[INFO] No model found. Please register at least one user first.")
            return

        cap = cv2.VideoCapture(self.CAMERA_INDEX)
        print("[INFO] Attendance system started. Press 'q' to quit.")

        while True:
            now = datetime.now()
            if now.time() >= self.LOGOUT_TIME:
                break

            ret, frame = cap.read()
            if not ret:
                continue

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = self.face_mesh.process(rgb)

            if result.multi_face_landmarks:
                landmarks = result.multi_face_landmarks[0].landmark
                vector = self.preprocess(landmarks)
                X = pd.DataFrame([vector], columns=self.features)

                try:
                    probs = self.model.predict_proba(X)[0]
                    max_prob = np.max(probs)
                    name = self.model.classes_[np.argmax(probs)]
                except:
                    max_prob = 0
                    name = None

                if max_prob < 0.7 or name not in self.model.classes_:
                    cv2.putText(frame, "Unknown Face - Please Register", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.imshow("Face Attendance - Mesh View", frame)
                    continue

                if name not in self.active_users:
                    self.mark_login(name, now)
                else:
                    self.active_users[name] = now

                mp.solutions.drawing_utils.draw_landmarks(
                    frame, result.multi_face_landmarks[0],
                    mp.solutions.face_mesh.FACEMESH_TESSELATION,
                    self.drawing_spec, self.drawing_spec
                )
                cv2.putText(frame, f"{name} Detected", (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            for user in list(self.active_users.keys()):
                if (now - self.active_users[user]).total_seconds() > self.TIMEOUT_SECONDS:
                    if user in self.login_times:
                        logout_time = now
                        login_time = self.login_times[user]
                        self.update_attendance_log(user, login_time, logout_time)
                        print(f"[TIMEOUT LOGOUT] {user} at {logout_time.strftime('%H:%M:%S')} | Duration: {logout_time - login_time}")
                        del self.login_times[user]
                    else:
                        print(f"[SKIPPED LOGOUT] {user} has no login record.")
                    del self.active_users[user]

            cv2.imshow("Face Attendance - Mesh View", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        now = datetime.now()
        for user in list(self.active_users.keys()):
            if user in self.login_times:
                login_time = self.login_times[user]
                self.update_attendance_log(user, login_time, now)
                print(f"[FORCED LOGOUT] {user} at {now.strftime('%H:%M:%S')} | Duration: {now - login_time}")
                del self.login_times[user]
        cap.release()
        cv2.destroyAllWindows()


# Main Entry Point
if __name__ == "__main__":  # Fixed this line
    system = FaceAttendanceSystem()

    while True:
        print("\n--- Face Attendance System ---")
        print("1. Register a New User")
        print("2. Start Attendance")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ").strip()

        if choice == '1':
            name = input("Enter name to register: ").strip()
            if name:
                system.register_user(name)
        elif choice == '2':
            system.start_attendance()
        elif choice == '3':
            print("Exiting Attendance System.")
            break
        else:
            print("Invalid choice. Please try again.")


--- Face Attendance System ---
1. Register a New User
2. Start Attendance
3. Exit


Enter your choice (1/2/3):  1
Enter name to register:  bhagya


[INFO] Registering: bhagya


[I 2025-07-15 11:22:24,828] A new study created in memory with name: no-name-e3e42a21-b976-4ffc-bf9f-2064ec532ed1
[I 2025-07-15 11:22:26,392] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 1.0.
[I 2025-07-15 11:22:28,633] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 150, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 1.0.
[I 2025-07-15 11:22:30,211] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 1.0.
[I 2025-07-15 11:22:31,096] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 50, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 1.0.
[I 2025-07-15 11:22:32,588] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 100, 

[SUCCESS] bhagya registered successfully.

--- Face Attendance System ---
1. Register a New User
2. Start Attendance
3. Exit


Enter your choice (1/2/3):  2


[INFO] Attendance system started. Press 'q' to quit.
[LOGIN] bhagya at 11:23:23 | Status: Late
[LOGOUT] bhagya at 11:23:30 | Duration: 0:00:06.486941 | Status: Late
[FORCED LOGOUT] bhagya at 11:23:30 | Duration: 0:00:06.486941

--- Face Attendance System ---
1. Register a New User
2. Start Attendance
3. Exit


Enter your choice (1/2/3):  3


Exiting Attendance System.
